In [1]:
import os
from skimage.measure import regionprops
from skimage.segmentation import slic
from skimage.util import img_as_float
from skimage.segmentation import mark_boundaries
from skimage import io
from skimage import color
import numpy as np
import argparse
import cv2
import sys
import random
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm


In [2]:
rootdir = 'D:\imagenet\\full\\train'
savedir = 'D:\imagenet\\full\\train_spixel'

image_paths = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        image_paths.append(os.path.join(subdir, file))

In [4]:
os.makedirs(savedir, exist_ok=True)
img_size = [int(256*1.15), int(256*1.15)]

for i in tqdm(range(len(image_paths))):
    path = image_paths[i]
    img = Image.open(path).resize(img_size).convert('RGB')
    img = np.array(img)
    fclass = path.split('\\')[-2]
    fname = path.split('\\')[-1].split('.')[0]
    converted_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

    height,width,channels = converted_img.shape
    num_iterations = 6
    prior = 2
    double_step = False
    num_superpixels = 256
    num_levels = 4
    num_histogram_bins = 5

    seeds = cv2.ximgproc.createSuperpixelSEEDS(width, height, channels, num_superpixels, num_levels, prior, num_histogram_bins)
    color_img = np.zeros((height, width, 3), np.uint8)
    color_img[:] = (0, 0, 255)
    seeds.iterate(converted_img, num_iterations)

    # retrieve the segmentation result
    labels = seeds.getLabels()
    quantize_img = img.copy()
    low = np.min(labels)
    high = np.max(labels)
    for c in range(low, high+1):
        quantize_img[labels == c, :] = np.mean(img[labels == c, :], axis=0)

    result = Image.fromarray(quantize_img)
    os.makedirs(os.path.join(savedir, fclass), exist_ok=True)
    result.save(os.path.join(savedir, fclass, fname+'.png'))


'''# Draw boundary contour
mask = seeds.getLabelContourMask(False)
mask_inv = cv2.bitwise_not(mask)
result_bg = cv2.bitwise_and(img, img, mask=mask_inv)
result_fg = cv2.bitwise_and(color_img, color_img, mask=mask)
result = cv2.add(result_bg, result_fg)

#Image.fromarray(result).show()'''

100%|██████████| 1281167/1281167 [30:49:32<00:00, 11.54it/s]


'# Draw boundary contour\nmask = seeds.getLabelContourMask(False)\nmask_inv = cv2.bitwise_not(mask)\nresult_bg = cv2.bitwise_and(img, img, mask=mask_inv)\nresult_fg = cv2.bitwise_and(color_img, color_img, mask=mask)\nresult = cv2.add(result_bg, result_fg)\n\n#Image.fromarray(result).show()'

In [ ]:
img = Image.open(os.path.join(savedir, 'n01440764', 'ILSVRC2012_val_00000293.png'))

array = np.array(img)
print(len(np.unique(array.reshape(-1, 3), axis=0)))

In [ ]:
numSegments = 256

for path in image_paths[1:2]:
    image = io.imread(path)
    if len(image.shape) == 2:
        image = color.gray2rgb(image)

    Image.fromarray((image).astype(np.uint8)).show()

    segments = slic(image, n_segments=256, compactness=10.0)

    regions = regionprops(segments, intensity_image=image)

    for i, r in enumerate(regions):
        print(i)
        image[r.coords[:, 0], r.coords[:, 1], :] = r.mean_intensity
    
    #image = mark_boundaries(image, segments)

Image.fromarray((image).astype(np.uint8)).show()


In [ ]:
img = cv2.imread(image_paths[0])

slic = cv2.ximgproc.createSuperpixelSLIC(img)
slic.iterate(10)
mask = slic.getLabelContourMask()

Image.fromarray(mask).show()